In [6]:
import sys
import datetime
import time

In [7]:
print sys.executable

/Users/romainbui/miniconda2/envs/mypython/bin/python


In [8]:
from rdkit.Chem import AllChem

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import pickle

In [11]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("Data/train")
df_test = pd.read_csv("Data/test")


In [12]:
def smiles_to_fingerprint (smiles, method='Morgan'):
    if method == "Morgan": 
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
        fp_arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, fp_arr)
        return fp_arr
    elif method == "other":
        print "another fingerprint"   

In [13]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [14]:
def do_forest(forest, smiles, y, fold=None):
    if fold:
        smiles = smiles[fold]
        y = y[fold]
    fps = smiles.apply(smiles_to_fingerprint).tolist()
    X = np.ndarray((smiles.shape[0], 2048))
    for i in range (smiles.shape[0]):
        X[i] = fps[i]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    forest.fit(X_train, y_train)
    print mean_squared_error(y_test, forest.predict(X_test))

In [15]:
#convert the test set to fingerprints for submission
test_fps = df_test.smiles.apply(smiles_to_fingerprint)
X_test = np.ndarray((824230, 2048))
for i in range (test_fps.shape[0]):
    X_test[i] = test_fps[i]
np.save('Data/test_morgans',X_test)

In [16]:
# Romain's Data
#test_r = pd.read_csv("Data/X_test_r.csv")
#train_r = pd.read_csv("Data/X_train_r.csv")

In [17]:
r = pd.read_csv("Data/df_train_OB2.csv")
r.index = r.smiles.values

In [18]:
r_test = pd.read_csv("Data/df_test_r.csv")
r_test.index = r_test.smiles.values

In [19]:
df_train.loc[:, 'C2'] = r.loc[df_train.smiles.values, 'C2'].values
df_train.loc[:, 'C3'] = r.loc[df_train.smiles.values, 'C3'].values
df_train.loc[:, 'Car'] = r.loc[df_train.smiles.values, 'Car'].values
df_train.loc[:, 'H'] = r.loc[df_train.smiles.values, 'H'].values
df_train.loc[:, 'N3'] = r.loc[df_train.smiles.values, 'N3'].values
df_train.loc[:, 'Nar'] = r.loc[df_train.smiles.values, 'Nar'].values
df_train.loc[:, 'O2'] = r.loc[df_train.smiles.values, 'O2'].values
df_train.loc[:, 'O3'] = r.loc[df_train.smiles.values, 'O3'].values
df_train.loc[:, 'S2'] = r.loc[df_train.smiles.values, 'S2'].values
df_train.loc[:, 'S3'] = r.loc[df_train.smiles.values, 'S3'].values
df_train.loc[:, 'Se'] = r.loc[df_train.smiles.values, 'Se'].values
df_train.loc[:, 'Si'] = r.loc[df_train.smiles.values, 'Si'].values
df_train.loc[:, 'weight'] = r.loc[df_train.smiles.values, 'weight'].values

In [20]:
df_test.loc[:, 'C2'] = r_test.loc[df_test.smiles.values, 'C2'].values
df_test.loc[:, 'C3'] = r_test.loc[df_test.smiles.values, 'C3'].values
df_test.loc[:, 'Car'] = r_test.loc[df_test.smiles.values, 'Car'].values
df_test.loc[:, 'H'] = r_test.loc[df_test.smiles.values, 'H'].values
df_test.loc[:, 'N3'] = r_test.loc[df_test.smiles.values, 'N3'].values
df_test.loc[:, 'Nar'] = r_test.loc[df_test.smiles.values, 'Nar'].values
df_test.loc[:, 'O2'] = r_test.loc[df_test.smiles.values, 'O2'].values
df_test.loc[:, 'O3'] = r_test.loc[df_test.smiles.values, 'O3'].values
df_test.loc[:, 'S2'] = r_test.loc[df_test.smiles.values, 'S2'].values
df_test.loc[:, 'S3'] = r_test.loc[df_test.smiles.values, 'S3'].values
df_test.loc[:, 'Se'] = r_test.loc[df_test.smiles.values, 'Se'].values
df_test.loc[:, 'Si'] = r_test.loc[df_test.smiles.values, 'Si'].values
df_test.loc[:, 'weight'] = r_test.loc[df_test.smiles.values, 'weight'].values

In [21]:
%time
from sklearn.ensemble import AdaBoostRegressor
print(datetime.datetime.now())
#forest = AdaBoostRegressor(n_estimators = 350)
forest = RandomForestRegressor(n_estimators=200, warm_start=True)
do_forest(forest, df_train.smiles,df_train.gap, slice(0,100000))
print(datetime.datetime.now())

CPU times: user 5 µs, sys: 8 µs, total: 13 µs
Wall time: 25 µs
2016-02-12 00:45:54.025306
0.00771984469583
2016-02-12 01:29:59.812632


In [22]:
f = open('Data/RF_OpenBabel.forest','wb')
pickle.dump(forest, f)
f.close()
#forest = pickle.load(f)

In [23]:
f = open('Data/RF_OpenBabel.forest','rb')
forest = pickle.load(f)

In [24]:
write_to_file('Data/RF_OpenBabel.csv', forest.predict(X_test))